In [1]:
import xarray as xr
import xradar as xd
import datatree
import matplotlib.pyplot as plt
from dask.distributed import Client, LocalCluster
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from pandas import to_datetime
from datetime import datetime
from functools import reduce
from matplotlib import pyplot
from datatree import DataTree, open_datatree
from matplotlib.animation import FuncAnimation
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from xmovie import Movie
from IPython.display import HTML
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
from xmovie import Movie
warnings.simplefilter("ignore")

/data/keeling/a/alfonso8/miniconda3/envs/xradar/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cluster = LocalCluster()  
client = Client(cluster)
cluster

LocalCluster(903e21ba, 'tcp://127.0.0.1:40096', workers=8, threads=48, memory=48.00 GiB)

In [3]:
path = '/data/keeling/a/alfonso8/accp/zarr_radar/TAB.zarr'

In [4]:
%%time
dt = open_datatree(f"{path}", engine='zarr')

CPU times: user 1.32 s, sys: 1.31 s, total: 2.63 s
Wall time: 6.18 s


In [5]:
ds = dt['sweep_1'].ds.drop_isel(times=568)
ds = ds#.sel(azimuth=slice(140, 170), range=slice(60*1e3, 120*1e3))

In [6]:
def custom_plotfunc(ds, fig, t0, *args, **kwargs):
    from pyart.graph import cm
    # Define station location for timeseries
    ds_f = ds.isel(times=t0)
    (ax, ax1) = fig.subplots(1, 2, subplot_kw={"projection": ccrs.PlateCarree()})
    proj_crs = xd.georeference.get_crs(ds)
    cart_crs = ccrs.Projection(proj_crs)
    sc = ds_f.DBZH.plot(x="x", y="y", vmin=-10,
                                vmax=60, cmap="pyart_ChaseSpectral", 
                                transform=cart_crs,
                                ax=ax)
    ax.set_title("")
    hour = pd.to_datetime(ds_f.times.values).tz_localize('UTC').tz_convert('America/Bogota')
    title = fig.suptitle(f"Tablazo radar - {ds_f.sweep_fixed_angle.values: .1f} [deg]    {hour:%Y-%m-%d %H:%M} HLC")
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color="gray", alpha=0.3, linestyle="--")
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    gl.top_labels = False
    gl.right_labels = False
    shape_feature = ShapelyFeature(Reader("./gis/shape/Departamentos.shp").geometries(),
                                    ccrs.PlateCarree(), facecolor='none', edgecolor='w')
    mun = ShapelyFeature(Reader("./gis/quet/muni.shp").geometries(),
                                    ccrs.PlateCarree(), facecolor='none', edgecolor='w')
    
    dren = ShapelyFeature(Reader("./gis/dren/Drenajes.shp").geometries(),
                                    ccrs.PlateCarree(), facecolor='none', edgecolor='w')
    
    sen = ShapelyFeature(Reader("./gis/dren/sencillo.shp").geometries(),
                                    ccrs.PlateCarree(), facecolor='none', edgecolor='w')
    # ax.add_feature(mun, edgecolor='c')
    ax.add_feature(shape_feature, edgecolor='w')
    # ax.add_feature(dren, edgecolor='blue', lw=0.4)
    # ax.add_feature(sen, edgecolor='blue', lw=0.4)
    
    ax.scatter(-73.8726, 4.3326, transform=ccrs.PlateCarree(), c='#00FF00', s=8, zorder=10)
    ax.set_aspect('equal', 'datalim')

    z_lin = 10 ** (0.1 *ds.isel(times=slice(0, t0)).DBZH)
    r_total = ((0.017 * z_lin ** 0.71428) * (5 / 60)).sum('times')
    p = ax1.pcolormesh(ds.x.values, ds.y.values, r_total,  transform=cart_crs, vmin=0, vmax=50, cmap='magma')
    gl1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color="gray", alpha=0.3, linestyle="--")
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    gl1.top_labels = False
    gl1.right_labels = False
    gl1.left_labels = False
    ax1.set_aspect('equal', 'datalim')
    # ax1.add_feature(mun, edgecolor='c')
    # ax1.add_feature(sen, edgecolor='b', lw=0.4)
    # ax1.add_feature(dren, edgecolor='blue', lw=0.4)
    ax1.add_feature(shape_feature, edgecolor='w')
    plt.colorbar(p, ax=ax1, label='Total rainfall rate [mm]')
    ax1.scatter(-73.8726, 4.3326, transform=ccrs.PlateCarree(), c='#00FF00', s=8, zorder=10)
    ax1.set_aspect('equal', 'datalim')


In [7]:
mov_custom = Movie(ds.chunk({'times':1}), custom_plotfunc, framedim='times', input_check=False, pixelwidth=1920, pixelheight=720, dpi=150)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [8]:
%%time
mov_custom.save(
    './results/movie_parallel_subset.mov',
    parallel=True,
    overwrite_existing=True,
)

/data/keeling/a/alfonso8/miniconda3/envs/xradar/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(
/data/keeling/a/alfonso8/miniconda3/envs/xradar/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(
/data/keeling/a/alfonso8/miniconda3/envs/xradar/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This 

Movie created at movie_parallel_subset.mov
CPU times: user 13min 38s, sys: 57.8 s, total: 14min 36s
Wall time: 19min 9s


In [9]:
# !rm f'./results/quetame1.mp4'
# ani.save(f'./results/quetame1.mp4')